<a href="https://colab.research.google.com/github/CamilleGreen5/Deep_Learning/blob/master/mobilenet_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import time
import glob
import os
import datetime
import pandas as pd

## Download Data

In [ ]:
# !wget http://www.image-net.org/challenges/LSVRC/2012/dd31405981ef5f776aa17412e1f0c112/ILSVRC2012_img_val.tar
# !wget https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt

In [ ]:
# !rm -r sample_data
# !mkdir ./imagenet
# !mkdir ./imagenet/train
# !mkdir ./imagenet/val
# !tar xf ILSVRC2012_img_val.tar -C ./imagenet/

In [ ]:
list_image = glob.glob("./imagenet/ILSVRC2012_val_*")
list_train_image = list_image[:35000]
list_val_image = list_image[35000:]
name_root = './imagenet/'
for image in list_train_image:
  name_image = image[11:]
  path = name_root + 'train/' + name_image
  os.rename(image, path)
for image in list_val_image:
  name_image = image[11:]
  path = name_root + 'val/' + name_image
  os.rename(image, path)

list_train_image = glob.glob("./imagenet/train/*")
list_val_image = glob.glob("./imagenet/val/*")
print(len(list_train_image))
print(len(list_val_image))

35000
15000


In [ ]:
list_label_train = []
list_label_val = []
f = open("imagenet_val_labels.txt", "r")
k=0

for line in f:
  if k < 35000:
    # label = int(line[0:-1])
    label = line[0:-1]
    list_label_train.append(label)
    k+=1
  elif 35000 <= k < 50000:
    # label = int(line[0:-1])
    label = line[0:-1]
    list_label_val.append(label)
    k+=1
f.close()

print(len(list_label_train))
print(len(list_label_val))

# array_label_val = np.zeros((len(list_label_val), 1001))
# array_label_train = np.zeros((len(list_label_train), 1001))
# for i, label in enumerate(list_label_val):
#   array_label_val[i, label] = 1
# for i, label in enumerate(list_label_train):
#   array_label_train[i, label] = 1

# print(array_label_train.shape)
# print(array_label_val.shape)

35000
15000


In [ ]:
list_train_image = np.sort(list_train_image)
list_val_image = np.sort(list_val_image)

train_dataset  = np.stack((list_train_image, list_label_train), axis=1)
val_dataset  = np.stack((list_val_image, list_label_val), axis=1)

train_dataset = pd.DataFrame(train_dataset, columns = ['path', 'label'])
val_dataset = pd.DataFrame(val_dataset, columns = ['path', 'label'])

train_dataset

,path,label
0,./imagenet/train/ILSVRC2012_val_00000001.JPEG,490
1,./imagenet/train/ILSVRC2012_val_00000003.JPEG,361
2,./imagenet/train/ILSVRC2012_val_00000005.JPEG,171
3,./imagenet/train/ILSVRC2012_val_00000007.JPEG,822
4,./imagenet/train/ILSVRC2012_val_00000010.JPEG,297
...,...,...
34995,./imagenet/train/ILSVRC2012_val_00049995.JPEG,67
34996,./imagenet/train/ILSVRC2012_val_00049997.JPEG,184
34997,./imagenet/train/ILSVRC2012_val_00049998.JPEG,454
34998,./imagenet/train/ILSVRC2012_val_00049999.JPEG,879


## Model

In [ ]:
%reload_ext tensorboard

In [ ]:
class MobilenetLayer(tf.keras.Model):

  def __init__(self, input_shape, filters, strides):
    
    super(MobilenetLayer, self).__init__()
    
    self.InputShape = input_shape
    self.Filters = filters
    self.Strides = strides

    self.inputs = tf.keras.layers.InputLayer(self.InputShape)
    self.depthconv = tf.keras.layers.DepthwiseConv2D((3, 3), strides=self.Strides, padding='same', depth_multiplier=1)
    self.bn1 = tf.keras.layers.BatchNormalization()
    self.activation = tf.keras.layers.Activation('relu')
    self.pointconv = tf.keras.layers.Conv2D(self.Filters, (1, 1), padding='same')
    self.bn2 = tf.keras.layers.BatchNormalization()
    self.outputs = tf.keras.layers.Activation('relu')
  
  def init_weights():
    official_model = tf.keras.applications.MobileNet((224, 224, 3))


  def call(self, input):
    x = self.inputs(input)
    x = self.depthconv(x)
    x = self.bn1(x)
    x = self.activation(x)
    x = self.pointconv(x)
    x = self.bn2(x)
    return self.outputs(x)

In [ ]:
# name / filters / strides / input_shape / output_shape
arch = np.array([
  ['MobilenetLayer0', 64, np.array([1,1]), np.array([112,112,32])],
  ['MobilenetLayer1', 128, np.array([2,2]), np.array([112,112,64])],
  ['MobilenetLayer2', 128, np.array([1,1]), np.array([56,56,128])],
  ['MobilenetLayer3', 256, np.array([2,2]), np.array([56,56,128])],
  ['MobilenetLayer4', 256, np.array([1,1]), np.array([28,28,256])],
  ['MobilenetLayer5', 512, np.array([2,2]), np.array([28,28,256])],
  ['MobilenetLayer6', 512, np.array([1,1]), np.array([14,14,512])],
  ['MobilenetLayer7', 1024, np.array([2,2]), np.array([14,14,512])],
  ['MobilenetLayer8', 1024, np.array([1,1]), np.array([7,7,1024])]
])


In [ ]:
class Mobilenet(tf.keras.Model):

  def __init__(self, arch, input_shape):
    
    super(Mobilenet, self).__init__()

    self.InputShape = input_shape

    self.inputs = tf.keras.layers.InputLayer(self.InputShape)
    self.conv2d = tf.keras.layers.Conv2D(32, (3, 3), padding='same', strides=(2, 2))
    self.bn1 = tf.keras.layers.BatchNormalization()
    self.activation = tf.keras.layers.Activation('relu')

    self.MobilenetLayer0 = MobilenetLayer(arch[0, 3], arch[0, 1], arch[0, 2])
    self.MobilenetLayer1 = MobilenetLayer(arch[1, 3], arch[1, 1], arch[1, 2])
    self.MobilenetLayer2 = MobilenetLayer(arch[2, 3], arch[2, 1], arch[2, 2])
    self.MobilenetLayer3 = MobilenetLayer(arch[3, 3], arch[3, 1], arch[3, 2])
    self.MobilenetLayer4 = MobilenetLayer(arch[4, 3], arch[4, 1], arch[4, 2])
    self.MobilenetLayer5 = MobilenetLayer(arch[5, 3], arch[5, 1], arch[5, 2])
    self.MobilenetLayer61 = MobilenetLayer(arch[6, 3], arch[6, 1], arch[6, 2])
    self.MobilenetLayer62 = MobilenetLayer(arch[6, 3], arch[6, 1], arch[6, 2])
    self.MobilenetLayer63 = MobilenetLayer(arch[6, 3], arch[6, 1], arch[6, 2])
    self.MobilenetLayer64 = MobilenetLayer(arch[6, 3], arch[6, 1], arch[6, 2])
    self.MobilenetLayer65 = MobilenetLayer(arch[6, 3], arch[6, 1], arch[6, 2])
    self.MobilenetLayer66 = MobilenetLayer(arch[6, 3], arch[6, 1], arch[6, 2])
    self.MobilenetLayer7 = MobilenetLayer(arch[7, 3], arch[7, 1], arch[7, 2])
    self.MobilenetLayer8 = MobilenetLayer(arch[8, 3], arch[8, 1], arch[8, 2])

    self.Pool = tf.keras.layers.AveragePooling2D(pool_size=(7, 7))
    self.Dense = tf.keras.layers.Dense(1000)
    self.Soft = tf.keras.layers.Activation('softmax')
    self.Shape = tf.keras.layers.Reshape((1000,))

  
  def call(self, input):
    x = self.inputs(input)
    x = self.conv2d(x)
    x = self.bn1(x)
    x = self.activation(x)
    x = self.MobilenetLayer0(x)
    x = self.MobilenetLayer1(x)
    x = self.MobilenetLayer2(x)
    x = self.MobilenetLayer3(x)
    x = self.MobilenetLayer4(x)
    x = self.MobilenetLayer5(x)
    x = self.MobilenetLayer61(x)
    x = self.MobilenetLayer62(x)
    x = self.MobilenetLayer63(x)
    x = self.MobilenetLayer64(x)
    x = self.MobilenetLayer65(x)
    x = self.MobilenetLayer66(x)
    x = self.MobilenetLayer7(x)
    x = self.MobilenetLayer8(x)
    x = self.Pool(x)
    x = self.Dense(x)
    x = self.Soft(x)
    x = self.Shape(x)
    return x

In [ ]:
input_shape = [224, 224, 3]

print("\nCREATE MODEL")
model = Mobilenet(arch, input_shape)

im = np.zeros((224, 224, 3), np.float32)
im = np.expand_dims(im, axis=0)
print('shape input image', im.shape)

print("\nCOMPUTE PRED")
t0 = time.time()
pred = model(im)
t1 = time.time()

print("time compute", t1-t0)
print('output shape', pred.shape)

model.summary()


CREATE MODEL
shape input image (1, 224, 224, 3)

COMPUTE PRED
time compute 0.6196391582489014
output shape (1, 1000)
Model: "mobilenet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              multiple                  896       
_________________________________________________________________
batch_normalization (BatchNo multiple                  128       
_________________________________________________________________
activation (Activation)      multiple                  0         
_________________________________________________________________
mobilenet_layer (MobilenetLa multiple                  2816      
_________________________________________________________________
mobilenet_layer_1 (Mobilenet multiple                  9728      
_____

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics='accuracy')

## Training

In [ ]:
def data_generator(mode='train'):

    img_gen = tf.keras.preprocessing.image.ImageDataGenerator()

    if mode == 'train':
        samples_and_labels = img_gen.flow_from_dataframe(train_dataset, None, x_col='path', y_col='label', target_size=(224, 224), batch_size=10, color_mode='rgb', class_mode='categorical')
    elif mode == 'val':
        samples_and_labels = img_gen.flow_from_dataframe(val_dataset, None, x_col='path', y_col='label', target_size=(224, 224), batch_size=10, color_mode='rgb', class_mode='categorical')
    else:
        print("chose a mode")

    for sample, label in samples_and_labels:
        yield(sample, label)
        
def create_train_generator():
    gen = data_generator('train')
    return gen
        
def create_val_generator():
    gen = data_generator('val')
    return gen

In [ ]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
train_dataset_generator = tf.data.Dataset.from_generator(create_train_generator, output_types=(tf.float32, tf.float32), output_shapes=((None, 224, 224, 3), (None,1000)))
validation_dataset_generator = tf.data.Dataset.from_generator(create_val_generator, output_types=(tf.float32, tf.float32), output_shapes=((None, 224, 224, 3), (None,1000)))


model.fit(train_dataset_generator, epochs=10, shuffle=True, callbacks=[tensorboard_callback], \
          steps_per_epoch=2000, validation_data=validation_dataset_generator, validation_steps=1000, verbose=2)

# model.save('./model/mymodel')
# print('model saved')

In [ ]:
gen = create_train_generator()
k = 0
for sample, label in gen:
  print(label)
  k += 1
  if k == 1:
    break

## Test

In [ ]:
list_official_model_layers = []
official_model = tf.keras.applications.MobileNet((224, 224, 3))
for layer in official_model.layers:
  list_official_model_layers.append(layer.name)
print(len(list_official_model_layers))

93


In [ ]:
input_shape = [224, 224, 3]

print("\nCREATE MODEL")
model = Mobilenet(arch, input_shape)

im = np.zeros((224, 224, 3), np.float32)
im = np.expand_dims(im, axis=0)
print('shape input image', im.shape)

print("\nCOMPUTE PRED")
t0 = time.time()
pred = model(im)
t1 = time.time()

print("time compute", t1-t0)
print('output shape', pred.shape, '\n')

list_model_layers = []

for layer in model.layers:
  if layer.name[:15] == 'mobilenet_layer':
    for layer_in in layer.layers:
      list_model_layers.append(layer_in.name)
  else:
    list_model_layers.append(layer.name)




CREATE MODEL
shape input image (1, 224, 224, 3)

COMPUTE PRED
time compute 0.2900967597961426
output shape (1, 1000) 



In [ ]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
pd_tab = []
for i in range(min(len(list_model_layers), len(list_official_model_layers))):
  pd_tab.append([list_model_layers[i], list_official_model_layers[i]])
pd_dataframe = pd.DataFrame(pd_tab, columns = ['list_model_layers', 'list_official_model_layers'])
pd_dataframe